In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('weixin.csv',encoding='gbk')
data.head()

,小程序名称,评价人数,人气,好评率,分类,标签,作者,发布时间,小程序介绍,评分
0,摩拜单车,17,977582,90%,出行交通,交通出行火车票酒店飞机票,摩拜单车,2017-01-11,扫码即可骑走的共享单车,4.5
1,她厨美食,19,447498,98%,生活美食,世界美食中国美食知名美食流行美食外国美食美食天下食谱大全家常食谱菜谱食谱,考拉网络,2017-04-07,源自意大利COOK FUN的原生理念开发，专注于世界美食的分享制作。由国际资深美厨家推送，易...,4.9
2,糗事秀,14,398650,84%,社交社区,搞笑段子趣味阅读表情社交,糗事秀,2017-01-19,每天分享有趣的图文信息，笑话锦集，让您的生活每天多姿多彩，爆笑连连,4.2
3,网易漫画,8,291961,100%,阅读漫画,卡通在线漫画,网易漫画,2017-02-14,网易漫画，有态度的看漫画神器，精品国漫日漫免费看！独家签约国内漫画大神。,5.0
4,一家民宿+,4,281619,76%,旅游酒店住宿,民宿酒店住宿日租房自由行民宿,一家民宿,2017-01-28,全球华人民宿信息平台，出境游旅行必备。专注华人一站式旅行，轻松跨越语言障碍。站内查看完相关房...,3.8


In [3]:
data.columns = ['name', 'evaluation', 'popular', 'good_eval', 'category', 'slogan', 'author', 'time', 'introduction', 'grade']
data.columns

Index(['name', 'evaluation', 'popular', 'good_eval', 'category', 'slogan',
       'author', 'time', 'introduction', 'grade'],
      dtype='object')

In [4]:
import jieba

In [5]:
introduction = data.introduction.values.tolist()

In [6]:
intro_S = []
for line in introduction:
    current_segment = jieba.lcut(line)
    if len(current_segment) > 1 and current_segment != '\r\n': #换行符
        intro_S.append(current_segment)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 5.981 seconds.
Prefix dict has been built succesfully.


In [7]:
data_intro=pd.DataFrame({'intro_S':intro_S})
data['intro'] = data_intro

In [8]:
stopwords=pd.read_csv("stopwords.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')

In [9]:
def drop_stopwords(contents,stopwords):
    contents_clean = []
    #all_words = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            #all_words.append(word)
        contents_clean.append(line_clean)
    return contents_clean#,all_words

In [10]:
contents = data.intro.values.tolist()    
stopwords = stopwords.stopword.values.tolist()
intro_clean = drop_stopwords(contents,stopwords)

In [11]:
data_intro_clean=pd.DataFrame({'intro_clean':intro_clean})

In [12]:
x_train = data_intro_clean['intro_clean'].values

In [13]:
words = []
for line_index in range(len(x_train)):
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        words.append(' '.join(x_train[line_index]))
    except:
        print (line_index,word_index)  

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
vectorizer = CountVectorizer()
#计算个词语出现的次数
X = vectorizer.fit_transform(words)
#获取词袋中所有文本关键词
word = vectorizer.get_feature_names()
print(word)

['1000', '1024', '107', '12306', '12315', '1330', '1400', '141', '150', '154', '1596974915', '160', '180', '1855', '188', '1v1', '200', '2000', '20000', '2000p', '2003', '2006', '2007', '2008', '2017', '211', '21cake', '23', '24', '250', '288', '28lite', '30', '300', '3000', '330', '330ml', '333', '356', '360', '365', '3c', '3d', '3v3', '40', '400', '4000', '4006', '4008033707', '4008202293', '4009', '4a', '4g', '4s店', '500', '5000', '594', '597', '5a', '5a56', '5s', '5v5', '600', '700', '777', '7x24', '8264', '836509', '8786', '900', '9000', '9066', '95', '95530', '98', '99', '999', 'aa', 'ai', 'all', 'app', 'aqi', 'assbook', 'authenticator', 'a股', 'bbc', 'bbs', 'bczp', 'behance', 'billboard', 'biubiu', 'blackweed', 'bna', 'bnbtrip', 'bp', 'brandsh', 'byebye', 'camera360', 'care', 'cba', 'checkinboss', 'cn', 'cnn', 'co', 'coffeesalon', 'cook', 'cosme', 'dawnsail', 'deepcube', 'diy', 'dj', 'doctor', 'dribbble', 'easy', 'ebaby', 'elong', 'er', 'erp', 'eyepetizer', 'fantastrip', 'fashion

In [16]:
#查看词频结果
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [17]:
X_array=X.toarray()


In [18]:
X_array.shape

(1212, 5704)

In [19]:
sum(X_array[1])/len(words[1].split())

0.9142857142857143

In [20]:
data['特征词比例']=0

In [21]:
for i in range(1212):
    data['特征词比例'][i]=sum(X_array[i])/len(words[i].split())

E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
data['特征词比例'].head()

0    0
1    0
2    1
3    1
4    0
Name: 特征词比例, dtype: int64

In [23]:
data['特征词比例'].tail()

1207    0
1208    1
1209    1
1210    0
1211    0
Name: 特征词比例, dtype: int64

In [24]:
data.shape

(1212, 12)

In [25]:
data.columns

Index(['name', 'evaluation', 'popular', 'good_eval', 'category', 'slogan',
       'author', 'time', 'introduction', 'grade', 'intro', '特征词比例'],
      dtype='object')

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer
 
#类调用
transformer = TfidfTransformer()
print(transformer)
#将词频矩阵X统计成TF-IDF值



TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)


In [27]:
tfidf = transformer.fit_transform(X)
#查看数据结构 tfidf[i][j]表示i类文本中的tf-idf权重
print(tfidf.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [28]:
tfidf_toarray=tfidf.toarray()

In [29]:
tfidf_toarray[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [30]:
sum(tfidf_toarray[1])

5.3869435107547

In [31]:
sum(tfidf_toarray[0])

1.9849301616511599

In [32]:
sum(tfidf_toarray[2])

2.8850400540712613

In [33]:
sum(tfidf_toarray[3])

3.1114411225864154

In [34]:
sum(tfidf_toarray[1211])

3.7756788755365402

In [35]:
data['TF-IDF权重']=0.0
for i in range(1212):
    data['TF-IDF权重'][i] = float(sum(tfidf_toarray[i]))

E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
data['TF-IDF权重'][1]

5.3869435107547

In [37]:
data['介绍字数']=0
for i in range(1212):
    data['介绍字数'][i] = len(data['introduction'][i])

E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
data.columns

Index(['name', 'evaluation', 'popular', 'good_eval', 'category', 'slogan',
       'author', 'time', 'introduction', 'grade', 'intro', '特征词比例', 'TF-IDF权重',
       '介绍字数'],
      dtype='object')

In [39]:
data['发布时间'] = 0
data['发布时间'] = pd.to_datetime(data['time'])#把object变量转为datetime
data.head()

,name,evaluation,popular,good_eval,category,slogan,author,time,introduction,grade,intro,特征词比例,TF-IDF权重,介绍字数,发布时间
0,摩拜单车,17,977582,90%,出行交通,交通出行火车票酒店飞机票,摩拜单车,2017-01-11,扫码即可骑走的共享单车,4.5,"[扫码, 即可, 骑, 走, 的, 共享, 单车]",0,1.984930,11,2017-01-11
1,她厨美食,19,447498,98%,生活美食,世界美食中国美食知名美食流行美食外国美食美食天下食谱大全家常食谱菜谱食谱,考拉网络,2017-04-07,源自意大利COOK FUN的原生理念开发，专注于世界美食的分享制作。由国际资深美厨家推送，易...,4.9,"[源自, 意大利, COOK, , FUN, 的, 原生, 理念, 开发, ，, 专注, ...",0,5.386944,103,2017-04-07
2,糗事秀,14,398650,84%,社交社区,搞笑段子趣味阅读表情社交,糗事秀,2017-01-19,每天分享有趣的图文信息，笑话锦集，让您的生活每天多姿多彩，爆笑连连,4.2,"[每天, 分享, 有趣, 的, 图文, 信息, ，, 笑话, 锦集, ，, 让, 您, 的,...",1,2.885040,33,2017-01-19
3,网易漫画,8,291961,100%,阅读漫画,卡通在线漫画,网易漫画,2017-02-14,网易漫画，有态度的看漫画神器，精品国漫日漫免费看！独家签约国内漫画大神。,5.0,"[网易, 漫画, ，, 有, 态度, 的, 看, 漫画, 神器, ，, 精品, 国漫, 日漫...",1,3.111441,36,2017-02-14
4,一家民宿+,4,281619,76%,旅游酒店住宿,民宿酒店住宿日租房自由行民宿,一家民宿,2017-01-28,全球华人民宿信息平台，出境游旅行必备。专注华人一站式旅行，轻松跨越语言障碍。站内查看完相关房...,3.8,"[全球华人, 民宿, 信息, 平台, ，, 出境游, 旅行, 必备, 。, 专注, 华人, ...",0,3.874083,50,2017-01-28


In [41]:
#del data['feature']


In [42]:
del data['good_eval']

In [43]:
del data['time']

In [44]:
del data['intro']

In [45]:
data.shape

(1212, 12)

In [46]:
data.columns

Index(['name', 'evaluation', 'popular', 'category', 'slogan', 'author',
       'introduction', 'grade', '特征词比例', 'TF-IDF权重', '介绍字数', '发布时间'],
      dtype='object')

In [47]:
data.describe()

,evaluation,popular,grade,特征词比例,TF-IDF权重,介绍字数
count,1212.000000,1212.000000,1212.000000,1212.000000,1212.000000,1212.000000
mean,1.358086,13837.047855,1.938531,0.431518,3.216252,51.011551
std,2.997190,37842.923899,2.348405,0.495492,0.995476,45.627109
min,0.000000,710.000000,0.000000,0.000000,0.000000,4.000000
25%,0.000000,3625.250000,0.000000,0.000000,2.522977,24.000000
50%,0.000000,8986.000000,0.000000,0.000000,3.218555,45.000000
75%,1.000000,15339.500000,5.000000,1.000000,3.819009,60.000000
max,36.000000,977582.000000,5.000000,1.000000,9.745138,588.000000


In [ ]:
train=data.rename({'name':'小程序名称', 'evaluation':'评价人数', 'popular':'人气', 'category':'分类', 'slogan':'标签','author':'作者', 'introduction':'小程序介绍', 'grade':'评分'}, axis=1)

In [ ]:
train.head()

In [ ]:
train.to_csv('train.csv')